## Softmax

In [21]:
import numpy as np

In [19]:
vector = np.array([11231, 999, 123142])

In [28]:
vector = np.array([-10, 10, 24])

In [33]:
def softmax(vec):
    vec -= np.max(vec)
    exp = np.exp(vec)
    return exp / np.sum(exp)

In [32]:
softmax(vector)

array([1.71390701e-15, 8.31528028e-07, 9.99999168e-01])

## Assignment-04 基于维基百科的词向量构建

在本章，你将使用Gensim和维基百科获得你的第一批词向量，并且感受词向量的基本过程。

## Step-01: Download Wikipedia Chinese Corpus: https://dumps.wikimedia.org/zhwiki/20190720/

第一步：使用维基百科下载中文语料库

## Step-02: Using https://github.com/attardi/wikiextractor to extract the wikipedia corpus

第二步：使用python wikipedia extractor抽取维基百科的内容

## Step-03: Using gensim get word vectors: 

Reference: 

+ https://radimrehurek.com/gensim/models/word2vec.html
+ https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne

第三步：参考Gensim的文档和Kaggle的参考文档，获得词向量。 注意，你要使用Jieba分词把维基百科的内容切分成一个一个单词，然后存进新的文件中。然后，你需要用Gensim的LineSentence这个类进行文件的读取。

再训练成词向量Model.

In [197]:
import jieba
from gensim.models.word2vec import LineSentence
from gensim.test.utils import datapath
import re
import nltk
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
%matplotlib inline

In [102]:
from hanziconv import HanziConv

In [116]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [124]:
STOP_WORDS = nltk.corpus.stopwords.words()
with open('stopwords.txt', 'w', encoding='utf-8') as f:
    for each in STOP_WORDS:
        f.write(each+' ')

In [204]:
with open('wiki_00', 'r', encoding='utf-8') as f1:
    with open('wiki_00_cut', 'w', encoding='utf-8') as f2:
        for each_line in f1:
            if each_line.strip():
                temp_result = []
                to_simplified = HanziConv.toSimplified(each_line)
                get_char = re.findall('[\u4e00-\u9fa5]+', to_simplified)
                for each in get_char:
                    temp_result += list(jieba.cut(each))

                f2.write(' '.join(temp_result) + '\n')

In [205]:
sentences = LineSentence(datapath('D:\\Github\\NLP\\Artificial_Intelligence_for_NLP\\\
Week_04_0727\\Assignment\\wiki_00_cut'))

In [206]:
model = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4)
model.save('word2vec.model')

C:\Users\Administrator\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\Administrator\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [207]:
model.wv.similar_by_word('数学')

[('一个', 0.9999138116836548),
 ('系统', 0.9999109506607056),
 ('研究', 0.9999025464057922),
 ('操作系统', 0.9999009966850281),
 ('的', 0.9998984336853027),
 ('是', 0.9998968839645386),
 ('都', 0.9998937845230103),
 ('一些', 0.9998931884765625),
 ('科学', 0.999885082244873),
 ('领域', 0.9998775720596313)]

In [208]:
model.wv.similarity('西方', '数学')

0.99963886

In [209]:
model.wv.distance('西方', '数学')

0.0003611445426940918

In [223]:
import os

In [224]:
for x in os.listdir('D:\\Github\\NLP\\Artificial_Intelligence_for_NLP\\Week_04_0727\\data\\text'):
    print(x)

.ipynb_checkpoints
AA
AB
AC
AD
AE
AF
AG
AH
AI
AJ
AK
AL
AM
clean_data.ipynb


## Step-04: Using some words to test your preformance. 

第四步，测试同义词，找几个单词。

## Step-05: Using visualization tools: 
> https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne

第五步：使用Kaggle给出的T-SEN进行词向量的可视化。